# Getting Started with OpenAI: Earnings Call Summaries

To use the OpenAI API from Python, we need to install the OpenAI Python package with pip.

In [ ]:
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.7 MB/s eta 0:00:00


Once the openai package is installed, we need to import it.

In [ ]:
import openai

## OpenAI API Keys

To use the API, you need an API key. If you haven't already, sign up for OpenAI. Visit the [API page on openai.com](https://openai.com/api/) and sign up. Under your account, visit the API Keys page and click "Create Secret Key". OpenAI provides $18.00 of credits to start, which is enough to get started:









Now that we have an API key, we need to provide it to the OpenAI Python package. Since I want to input my API key without showing it here, I am going to import getpass and prompt myself for the API Key:

In [ ]:
from getpass import getpass
openai.api_key = getpass()

KeyboardInterrupt: ignored

### Blob Connection

In [ ]:
!pip install azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.4/185.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 117.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
!pip install langchain
!pip install tiktoken

In [ ]:
!pip install unstructured

In [ ]:
from langchain.document_loaders import AzureBlobStorageContainerLoader

loaders = AzureBlobStorageContainerLoader(conn_str="DefaultEndpointsProtocol=https;AccountName=openaiblob1;EndpointSuffix=core.windows.net",container="onefilecontainer")
documents = loaders.load()
print(f'You have {len(documents)} documents in your data')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


You have 1 documents in your data


### Split the Text to summarize from Blob

In [ ]:
text_to_summarize = documents[0].page_content
len(text_to_summarize)

248860

In [ ]:
prompt = "I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will give you the answer. If you ask me a question that is nonsense, trickery, or has no clear answer, I will respond with \"Unknown\". Q: Who is the greatest investor of all time?\n\n"
engine = 'text-davinci-003'
response = openai.Completion.create(
  engine=engine,
  prompt=prompt,
  temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
  max_tokens=140,
  top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
  frequency_penalty=0,
  presence_penalty=1
)

response

<OpenAIObject text_completion id=cmpl-7bZDl9PNPod7o4FUT7VI9Io5vaUss at 0x7b32a8b69df0> JSON: {
  "id": "cmpl-7bZDl9PNPod7o4FUT7VI9Io5vaUss",
  "object": "text_completion",
  "created": 1689187881,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\nThe greatest investor of all time is widely considered to be Warren Buffett.",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 66,
    "completion_tokens": 15,
    "total_tokens": 81
  }
}

In [ ]:
prompt = f"{text_to_summarize}\n\ntl;dr:"

prompt

In [ ]:
#TOKEN LIMIT - DON'T RUN
response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
    max_tokens=140,
    top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
    frequency_penalty=0,
    presence_penalty=1
)

In [ ]:
words = text_to_summarize.split(" ")
len(words)

37910

In [ ]:
import numpy as np
chunks = np.array_split(words, 15)
chunks

In [ ]:
sentences = ' '.join(list(chunks[0]))
sentences

"Contoso Electronics\n\nNorthwind Health Standard Plan\n\nThis document contains information generated using a language model (Azure OpenAI). The information contained in this document is only for demonstration purposes and does not reflect the opinions or beliefs of Microsoft. Microsoft makes no representations or warranties of any kind, express or implied, about the completeness, accuracy, reliability, suitability or availability with respect to the information contained in this document.\n\nAll rights reserved to Microsoft\n\nSummary of Benefits\n\nNorthwind Standard Northwind Standard is a basic plan that provides coverage for medical, vision, and dental services. This plan also offers coverage for preventive care services, as well as prescription drug coverage. With Northwind Standard, you can choose from a variety of in-network providers, including primary care physicians, specialists, hospitals, and pharmacies. This plan does not offer coverage for emergency services, mental hea

In [ ]:
#SUMMARY FOR ONE CHUNK
prompt = f"{sentences}\n\ntl;dr:"

response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
    max_tokens=140,
    top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
    frequency_penalty=0,
    presence_penalty=1
)

response_text = response["choices"][0]["text"]
response_text

'\nThis document contains information about the Northwind Standard health plan, which provides coverage for medical, vision, and dental services. It explains the various costs associated with this plan, such as premiums, deductibles, co-payments, and co-insurance. It also provides tips for managing your costs, such as taking advantage of preventive care services and visiting in-network providers. Additionally, it explains how providers affect your costs, such as the importance of continuity of care and balance billing protection. Finally, it outlines the benefits of using out-of-network or non-contracted providers.'

In [ ]:
summary_responses = []

for chunk in chunks:
    sentences = ' '.join(list(chunk))
    prompt = f"{sentences}\n\ntl;dr:"
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
        max_tokens=150,
        top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
        frequency_penalty=0,
        presence_penalty=1
    )

    response_text = response["choices"][0]["text"]
    summary_responses.append(response_text+'\n\n')

full_summary = "".join(summary_responses)

print("full summary")
print(full_summary)

full summary

This document contains information about the Northwind Standard health plan, which provides coverage for medical, vision, and dental services. It explains the various costs associated with this plan, such as premiums, deductibles, co-payments, and co-insurance. It also provides tips on how to manage your costs and outlines the differences between in-network and out-of-network providers. Finally, it explains the balance billing protection offered by Northwind Standard and provides tips on how to avoid unexpected costs.

 Northwind Standard has a calendar year deductible of $2,000 for individuals and $4,000 for families. The plan also includes copayments for certain services, coinsurance of 20% for in-network services and 40% for out-of-network services, and an out-of-pocket maximum of $6,350 for individuals and $12,700 for families. Certain services may not be subject to the out-of-pocket maximum, such as non-covered services or services provided outside of the Northwind H

In [ ]:
full_summary

" Northwind Standard is a basic health plan that provides coverage for medical, vision, and dental services. This plan offers coverage for preventive care services, as well as prescription drug coverage. Premiums, deductibles, co-payments, and co-insurance are all costs associated with this plan. In order to save money on health care costs, it is important to choose an in-network provider. Out-of-network providers may require balance billing, meaning you will be responsible for the difference between what they charge and what your insurance company is willing to pay. Balance billing protection is available when visiting in-network providers.\n\nThe Northwind Standard plan includes copayments (copays) for certain services, a calendar year deductible of $2,000 for each individual and $4,000 for each family, coinsurance set at 20% for in-network services and 40% for out-of-network services, and an out-of-pocket maximum of $6,350 for an individual and $12,700 for a family. It is important 

In [ ]:
len(full_summary)

8597

### SUMMARIZE full-summary even further

In [ ]:
import numpy as np
words2 = full_summary.split(" ")
chunks2 = np.array_split(words2, 3)
print("CHUNKS: \n",chunks[0])
print("CHUNKS2: \n",chunks2[0])
len(chunks2)

CHUNKS: 
 ['Contoso' 'Electronics\n\nNorthwind' 'Health' ... 'out-of-network'
 'providers.' 'If']
CHUNKS2: 
 ['\nThis' 'document' 'contains' 'information' 'about' 'the' 'Northwind'
 'Standard' 'health' 'plan,' 'which' 'provides' 'coverage' 'for'
 'medical,' 'vision,' 'and' 'dental' 'services.' 'It' 'explains' 'the'
 'various' 'costs' 'associated' 'with' 'this' 'plan,' 'such' 'as'
 'premiums,' 'deductibles,' 'co-payments,' 'and' 'co-insurance.' 'It'
 'also' 'provides' 'tips' 'on' 'how' 'to' 'manage' 'your' 'costs' 'and'
 'outlines' 'the' 'differences' 'between' 'in-network' 'and'
 'out-of-network' 'providers.' 'Finally,' 'it' 'explains' 'the' 'balance'
 'billing' 'protection' 'offered' 'by' 'Northwind' 'Standard' 'and'
 'provides' 'tips' 'on' 'how' 'to' 'avoid' 'unexpected' 'costs.\n\n'
 'Northwind' 'Standard' 'has' 'a' 'calendar' 'year' 'deductible' 'of'
 '$2,000' 'for' 'individuals' 'and' '$4,000' 'for' 'families.' 'The'
 'plan' 'also' 'includes' 'copayments' 'for' 'certain' 'services

3

In [ ]:
sentences2 = ' '.join(list(chunks2[0]))
sentences2

'\nThis document contains information about the Northwind Standard health plan, which provides coverage for medical, vision, and dental services. It explains the various costs associated with this plan, such as premiums, deductibles, co-payments, and co-insurance. It also provides tips on how to manage your costs and outlines the differences between in-network and out-of-network providers. Finally, it explains the balance billing protection offered by Northwind Standard and provides tips on how to avoid unexpected costs.\n\n Northwind Standard has a calendar year deductible of $2,000 for individuals and $4,000 for families. The plan also includes copayments for certain services, coinsurance of 20% for in-network services and 40% for out-of-network services, and an out-of-pocket maximum of $6,350 for individuals and $12,700 for families. Certain services may not be subject to the out-of-pocket maximum, such as non-covered services or services provided outside of the Northwind Health net

In [ ]:
import numpy as np
words2 = full_summary.split(" ")
chunks2 = np.array_split(words2, 3)
#sentences2 = ' '.join(list(chunks2[0]))
summary_responses2 = []

for chunk in chunks2:
    sentences2 = ' '.join(list(chunk))
    prompt2 = f"{sentences2}\n\ntl;dr:"
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt2,
        temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
        max_tokens=150,
        top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
        frequency_penalty=0,
        presence_penalty=1
    )

    response_text = response["choices"][0]["text"]
    summary_responses2.append(response_text+'\n\n')

full_summary2 = "".join(summary_responses2)

In [ ]:
print("full summary")
print(full_summary2)

full summary
 Northwind Standard is a health plan that provides coverage for medical, vision, and dental services. It has a calendar year deductible of $2,000 for individuals and $4,000 for families, copayments, coinsurance of 20% for in-network services and 40% for out-of-network services, and an out-of-pocket maximum of $6,350 for individuals and $12,700 for families. It covers cellular immunotherapy and gene therapy, foot care services, massage therapy services, newborn care, and spinal services. It does not cover home health care, long-term care, alternative therapies, or cosmetic treatments.

 Northwind Standard is a comprehensive health insurance plan that covers manipulation of the spine and other joints, as well as the soft tissues of the body. It also covers urgent care services when they are medically necessary and requires prior authorization for certain services and treatments. The plan includes personal health support programs to help manage chronic conditions, injuries, a

In [ ]:
len(full_summary2)

1929

In [ ]:
import numpy as np
count=0
def recursive_summarize(text_to_summarize, target_word_count=200):
    words = text_to_summarize.split(" ")
    chunks = np.array_split(words, 15)
    summary_responses = []
    full_summary = ""

    for chunk in chunks:
        sentences = ' '.join(list(chunk))
        prompt = f"{sentences}\n\ntl;dr:"
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            temperature=0.3,
            max_tokens=150,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=1
        )

        response_text = response["choices"][0]["text"]
        summary_responses.append(response_text+'\n\n')

    full_summary = "".join(summary_responses)
    word_count = len(full_summary.split())
    print(word_count)
    if word_count > target_word_count:
    #     print(++count)
        return recursive_summarize(full_summary, target_word_count)

    return full_summary

# Example usage
initial_text_to_summarize = documents[0].page_content
final_summary = recursive_summarize(initial_text_to_summarize, target_word_count=50)


1253


RateLimitError: ignored

In [ ]:
print(final_summary)
print(len(final_summary))


This document contains information about the Northwind Standard health plan, which provides coverage for medical, vision, and dental services. It explains the various costs associated with this plan, such as premiums, deductibles, co-payments, and co-insurance. It also provides tips on how to manage your costs and discusses the importance of choosing in-network providers. Additionally, it explains balance billing protection and the benefits of out-of-network or non-contracted providers.

 The Northwind Standard plan includes an out-of-pocket maximum of $6,350 for an individual and $12,700 for a family. This limit applies to certain covered services and includes deductibles, coinsurance, and copayments. Certain services, such as non-covered services or services provided outside of the Northwind Health network, are not subject to the out-of-pocket maximum. Additionally, the out-of-pocket maximum will reset at the start of the calendar year, so any expenses paid during the previous year 

## OpenAI Playground

Now that we have access to the OpenAI API and have configured the Python package, what can we do with it? If you have not already done so, I suggest trying some of the examples in the [OpenAI Playground](https://beta.openai.com/examples). This will allow you to experiment with various prompts and their responses without writing any code.

# Calling OpenAI From Python

While it is fun to talk to GPT-3 from the Playground, we will get the most value out of OpenAI if we know how to code. With a few lines of Python, we can dynamically craft specific prompts, programmatically call OpenAI, and process the outputs. By doing this we can obtain interesting results and create our own custom applications. Let's start with a simple Q & A prompt and ask GPT-3 a question:

In [ ]:
prompt = "I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will give you the answer. If you ask me a question that is nonsense, trickery, or has no clear answer, I will respond with \"Unknown\". Q: Who is the greatest investor of all time?\n\n"
engine = 'text-davinci-003'
response = openai.Completion.create(
  engine=engine,
  prompt=prompt,
  temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
  max_tokens=140,
  top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
  frequency_penalty=0,
  presence_penalty=1
)

response

<OpenAIObject text_completion id=cmpl-7bYyAa7iQhlIY5qRAuSvLuZVzJzQc at 0x7b32e0cf23e0> JSON: {
  "id": "cmpl-7bYyAa7iQhlIY5qRAuSvLuZVzJzQc",
  "object": "text_completion",
  "created": 1689186914,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\nThe greatest investor of all time is widely considered to be Warren Buffett.",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 66,
    "completion_tokens": 15,
    "total_tokens": 81
  }
}

OpenAI replies that Warren Buffett is widely considered the greatest investor of all time. Notice that we set a variety of parameters in our code above: engine, prompt, temperature, max_tokens, etc. A full list of the available models and parameters are available on the [OpenAI GPT-3 models page](https://beta.openai.com/docs/models/gpt-3). You should experiment with different models and observe their differences and capabilities. They have different costs associated with them, so if you are performing a simpler task than you should use a simpler and cheaper model.

# Summarization Task

In the previous tutorial we used OpenAI Whisper to transcribe the audio of a Fed Speech. We can do the same to obtain the text transcript of an earnings call. I have provided a [transcript of the most recent Nvidia earnings call here](https://gist.github.com/hackingthemarkets/e664894b65b31cbe8993e02d25d26768). As you can see, this is quite a long earnings call which could take over an hour to listen to. In addition, it would take a long time to extract the most pertinent information from this call. Let's see if we can feed this earnings call to GPT-3 and have it process the language. We can do this by using the Summarization / TLDR prompt.

First, let's read the earnings call text into a variable. We can use the requests library to fetch the raw text of the earnings call that I have linked above:

In [ ]:
import requests

url = "https://gist.githubusercontent.com/hackingthemarkets/e664894b65b31cbe8993e02d25d26768/raw/618afe09d07979cc72911ce79634ab5d2cc19a54/nvidia-earnings-call.txt"
response = requests.get(url)
transcript = response.text

transcript



Now that we have the earnings text in a data variable, let's make this into a summarization prompt. To do this, we can simply append "tldr;" and some carriage returns to the data:

In [ ]:
prompt = f"{transcript}\n\ntl;dr:"

prompt

"Operator\n\nGood afternoon. My name is Emma, and I will be your conference operator today. At this time, I would like to welcome everyone to the NVIDIA's third quarter earnings call. [Operator instructions] Simona Jankowski, you may begin your conference.\n\nSimona Jankowski -- Vice President, Investor Relations\n\nThank you. Good afternoon, everyone, and welcome to NVIDIA's conference call for the third quarter of fiscal 2023. With me today from NVIDIA are Jen-Hsun Huang, president and chief executive officer; and Colette Kress, executive vice president and chief financial officer. I'd like to remind you that our call is being webcast live on NVIDIA's investor relations website.\n\n\nThe webcast will be available for replay until the conference call to discuss our financial results for the fourth quarter and fiscal 2023. The content of today's call is NVIDIA's property. It can't be reproduced or transcribed without our prior written consent. During this call, we may make forward-look

Now that we have a prompt, let's call OpenAI using this prompt:

In [ ]:
response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
    max_tokens=140,
    top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
    frequency_penalty=0,
    presence_penalty=1
)

InvalidRequestError: ignored

## Chunking up our data

Notice that we receive an error. This is because the API can not accept the entire document at once: there is a token limit for the input. To get around this, let's process our data. We'll split up the document into smaller chunks, generate a prompt for each chunk, then ask OpenAI to summarize each chunk. We'll then collate / concatenate the responses into a single summary at the end.

There are many ways to slice and dice text. Let's try first splitting up our text into a list of words, then dividing that list of words into equal chunks using numpy.

In [ ]:
words = transcript.split(" ")

# show the first 20 words
words[:20]

['Operator\n\nGood',
 'afternoon.',
 'My',
 'name',
 'is',
 'Emma,',
 'and',
 'I',
 'will',
 'be',
 'your',
 'conference',
 'operator',
 'today.',
 'At',
 'this',
 'time,',
 'I',
 'would',
 'like']

Now let's use numpy's array_split function to divide our list of words into equal parts - I'm hardcoding the number 6 here to divide the text into 6 equal parts, but you can create special rules for dividing up the text as you see fit.


In [ ]:
import numpy as np

chunks = np.array_split(words, 6)

chunks

[array(['Operator\n\nGood', 'afternoon.', 'My', ..., 'total', 'number',
        'of'], dtype='<U28'),
 array(['RTX', 'games', 'and', ..., 'path', 'forward', 'as'], dtype='<U28'),
 array(['general', 'purpose', 'computing', ..., 'that', 'while', 'they'],
       dtype='<U28'),
 array(['can', 'still', 'get', ..., 'taking', 'the', 'question.'],
       dtype='<U28'),
 array(['I', 'want', 'to', ..., 'Huang', '--', 'President'], dtype='<U28'),
 array(['and', 'Chief', 'Executive', ..., 'progress', 'next', 'quarter.'],
       dtype='<U28')]

As you can see, we now have 6 arrays of words. We can join each of these 6 chunks into their original text and pass each of them as a call to OpenAI to obtain a summary. We will save each summary response to a list, then join them all at the end into a single summary.

Before we call OpenAI 6 times, let's first walk through how a single chunk summary looks and make sure our first call works. Let's summarize the first chunk, chunks[0].



In [ ]:
sentences = ' '.join(list(chunks[0]))

sentences

"Operator\n\nGood afternoon. My name is Emma, and I will be your conference operator today. At this time, I would like to welcome everyone to the NVIDIA's third quarter earnings call. [Operator instructions] Simona Jankowski, you may begin your conference.\n\nSimona Jankowski -- Vice President, Investor Relations\n\nThank you. Good afternoon, everyone, and welcome to NVIDIA's conference call for the third quarter of fiscal 2023. With me today from NVIDIA are Jen-Hsun Huang, president and chief executive officer; and Colette Kress, executive vice president and chief financial officer. I'd like to remind you that our call is being webcast live on NVIDIA's investor relations website.\n\n\nThe webcast will be available for replay until the conference call to discuss our financial results for the fourth quarter and fiscal 2023. The content of today's call is NVIDIA's property. It can't be reproduced or transcribed without our prior written consent. During this call, we may make forward-look

In [ ]:

prompt = f"{sentences}\n\ntl;dr:"

response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
    max_tokens=140,
    top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
    frequency_penalty=0,
    presence_penalty=1
)

response_text = response["choices"][0]["text"]
response_text

"\n\nAt NVIDIA's third quarter earnings call, Colette Kress, executive vice president and chief financial officer, reported that revenue was $5.93 billion, down 12% sequentially and 17% year-on-year. Data center revenue of $3.83 billion was up 1% sequentially and 31% year-on-year, driven by leading U.S. cloud providers and a broadening set of consumer Internet companies. Gaming revenue of $1.57 billion was down 23% sequentially and 51% year-on-year, reflecting lower sell-in to partners to help align channel inventory levels with current demand expectations. The first ADA GPU, the GeForce RTX 4090"

This looks pretty good! Let's now loop through all of the chunks and save the responses in a list called summary_responses. At the end we'll join all of the responses into a single summary and check out the output.

In [ ]:
summary_responses = []

for chunk in chunks:

    sentences = ' '.join(list(chunk))

    prompt = f"{sentences}\n\ntl;dr:"

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.3, # The temperature controls the randomness of the response, represented as a range from 0 to 1. A lower value of temperature means the API will respond with the first thing that the model sees; a higher value means the model evaluates possible responses that could fit into the context before spitting out the result.
        max_tokens=150,
        top_p=1, # Top P controls how many random results the model should consider for completion, as suggested by the temperature dial, thus determining the scope of randomness. Top P’s range is from 0 to 1. A lower value limits creativity, while a higher value expands its horizons.
        frequency_penalty=0,
        presence_penalty=1
    )

    response_text = response["choices"][0]["text"]
    summary_responses.append(response_text)

full_summary = "".join(summary_responses)

print("full summary")
print(full_summary)



full summary


At NVIDIA's third quarter earnings call, Colette Kress, executive vice president and chief financial officer, reported that revenue was $5.93 billion, down 12% sequentially and 17% year-on-year. Data center revenue of $3.83 billion was up 1% sequentially and 31% year-on-year, driven by leading U.S. cloud providers and a broadening set of consumer Internet companies. Gaming revenue of $1.57 billion was down 23% sequentially and 51% year-on-year due to channel inventory corrections and external conditions. The RTX 4090 and 4080 GPUs were released with positive feedback from the gaming community.
In Q3, gaming sell-through was relatively solid, with notebook sales particularly strong. Desktop sales were softer in China and Asia. We expect stronger end demand in Q4 due to the upcoming holidays and continued AI adoption. In terms of data center, our business is indexed to two fundamental dynamics: the need for accelerated computing to achieve cost efficiency and energy effici

Pretty cool right? I know this is a simple example, but it lays a solid baseline for building more complex applications. We can already convert audio to text and convert text from one form to another. From this, we can build a web or mobile apps that accept sound files, video files, or URLs. We can then have these applications call OpenAI to perform complex tasks and return interesting results. With further creativity and additional code, we can build some very useful apps!
